In [ ]:
import pandas as pd
import gc
import numpy as np

from matplotlib import pyplot as plt
from sklearn.metrics import roc_auc_score
%matplotlib inline

# read dataset

In [2]:
train = pd.read_csv('train.csv')

C:\Users\RomanG\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (0,2) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
train.columns

Index(['C_Year', 'EtalonPosName', 'R_Year', 'age_qty', 'boss_pos_name',
       'boss_subord_1y_max_qty', 'boss_subord_1y_min_qty',
       'boss_subord_in_1m_qty', 'boss_subord_in_1y_qty',
       'boss_subord_out_1m_qty',
       ...
       'up_day_nv_sum_3m', 'up_day_nv_sum_6m', 'vl_nv_KBP_sum_3m',
       'vl_nv_KBP_sum_6m', 'vl_nv_MP_sum_3m', 'vl_nv_MP_sum_6m',
       'vl_nv_sum_3m', 'vl_nv_sum_6m', 'vsp_cd', 'work_rate_pct'],
      dtype='object', length=397)

In [4]:
test = pd.read_csv('test.csv')

In [5]:
valid_logins = pd.read_csv('valid_logins.csv')

# primary analysis

In [6]:
train['report_dt'] = pd.to_datetime(train['report_dt'])
valid_logins['business_dt'] = pd.to_datetime(valid_logins['business_dt'])
valid_logins['work_ends'] = pd.to_datetime(valid_logins['work_ends'])
valid_logins['work_begins'] = pd.to_datetime(valid_logins['work_begins'])

In [7]:
valid_logins['b_dt_month']=valid_logins['business_dt'].apply(lambda x: x.month)
valid_logins['b_dt_year']= valid_logins['business_dt'].apply(lambda x: x.year)
valid_logins['work_duration']=(valid_logins['work_ends']-valid_logins['work_begins']).apply(lambda x: x.seconds/3600)
valid_logins['work_ends_hour']=valid_logins['work_ends'].apply(lambda x: x.hour)
valid_logins['work_begins_hour']=valid_logins['work_begins'].apply(lambda x: x.hour)

In [8]:
train.shape

(449576, 397)

In [9]:
valid_logins.shape

(1291133, 10)

In [ ]:
month_agg = valid_logins.groupby(['tabnum','b_dt_month','b_dt_year']).agg(
    {
        'work_time_day':['mean','std', 'max', 'min'],
        'work_begins_hour': ['mean', 'std', 'max', 'min'],
        'work_ends_hour': ['mean', 'std', 'max', 'min'],
        'work_duration': ['mean', 'std','max', 'min']
    }
)

In [ ]:
month_agg

In [10]:
login_guys = valid_logins['tabnum'].unique()

def aggregate_logins(row):
    if row['tabnum'] not in login_guys:
        return None
    
    report_dt = row['report_dt']
    
    if report_dt.month not in list(month_agg.loc[(row['tabnum'])].index.get_level_values(level=0)):
        return pd.Series()
    if report_dt.year not in list(month_agg.loc[(row['tabnum'],report_dt.month)].index.get_level_values(level=0)):
        return pd.Series()
                                                
    a = month_agg.loc(axis=0)[(row['tabnum'],report_dt.month,report_dt.year)] 
    
    m3_ago = report_dt - pd.DateOffset(months=3)
    m3 = month_agg.query('tabnum == '+str(row['tabnum'])+' and b_dt_month >='+str(m3_ago.month)+' and b_dt_year >='+str(m3_ago.year)+' and b_dt_month <='+str(report_dt.month)+' and b_dt_year <='+str(report_dt.year)).mean()
    
    year_ago = report_dt - pd.DateOffset(years=1)
    y = month_agg.query('tabnum == '+str(row['tabnum'])+' and b_dt_month >='+str(year_ago.month)+' and b_dt_year >='+str(year_ago.year)+' and b_dt_month <='+str(report_dt.month)+' and b_dt_year <='+str(report_dt.year)).mean()
    
                                                
    result = pd.Series({
        'work_time_day_mean_3m':m3[('work_time_day', 'mean')],
        'work_time_day_std_3m':m3[('work_time_day', 'std')],
        'work_time_day_max_3m':m3[('work_time_day', 'max')],
        'work_time_day_min_3m':m3[('work_time_day', 'min')],
        'work_time_day_mean_year':y[('work_time_day', 'mean')],
        'work_time_day_std_year':y[('work_time_day', 'std')],
        'work_time_day_max_year':y[('work_time_day', 'max')],
        'work_time_day_min_year':y[('work_time_day', 'min')],
        
        'work_begins_hour_mean_3m':m3[('work_begins_hour', 'mean')],
        'work_begins_hour_std_3m':m3[('work_begins_hour', 'std')],
        'work_begins_hour_max_3m':m3[('work_begins_hour', 'max')],
        'work_begins_hour_min_3m':m3[('work_begins_hour', 'min')],
        'work_begins_hour_mean_year':y[('work_begins_hour', 'mean')],
        'work_begins_hour_std_year':y[('work_begins_hour', 'std')],
        'work_begins_hour_max_year':y[('work_begins_hour', 'max')],
        'work_begins_hour_min_year':y[('work_begins_hour', 'min')],
        
        'work_ends_hour_mean_3m':m3[('work_ends_hour', 'mean')],
        'work_ends_hour_std_3m':m3[('work_ends_hour', 'std')],
        'work_ends_hour_max_3m':m3[('work_ends_hour', 'max')],
        'work_ends_hour_min_3m':m3[('work_ends_hour', 'min')],
        'work_ends_hour_mean_year':y[('work_ends_hour', 'mean')],
        'work_ends_hour_std_year':y[('work_ends_hour', 'std')],
        'work_ends_hour_max_year':y[('work_ends_hour', 'max')],
        'work_ends_hour_min_year':y[('work_ends_hour', 'min')],
        
        'work_duration_mean_3m':m3[('work_duration', 'mean')],
        'work_duration_std_3m':m3[('work_duration', 'std')],
        'work_duration_max_3m':m3[('work_duration', 'max')],
        'work_duration_min_3m':m3[('work_duration', 'min')],
        'work_duration_mean_year':y[('work_duration', 'mean')],
        'work_duration_std_year':y[('work_duration', 'std')],
        'work_duration_max_year':y[('work_duration', 'max')],
        'work_duration_min_year':y[('work_duration', 'min')],
 
        
    })
    print(row['tabnum'])
    return result

In [ ]:
logins4train =train.apply(aggregate_logins,1)

In [ ]:
logins4train.shape

In [ ]:
logins4train.to_csv('logins4train.csv')

In [11]:
logins4train=pd.read_csv('logins4train.csv')

In [12]:
train = train.join(logins4train)

In [13]:
train['3M/Y_work_time_day_mean'] = train['work_time_day_mean_3m']/train['work_time_day_mean_year']
train['3M/Y_work_time_day_std'] = train['work_time_day_std_3m']/train['work_time_day_std_year']
train['3M/Y_work_time_day_max'] = train['work_time_day_max_3m']/train['work_time_day_max_year']
train['3M/Y_work_time_day_min'] = train['work_time_day_min_3m']/train['work_time_day_min_year']

train['3M/Y_work_begins_hour_mean'] = train['work_begins_hour_mean_3m']/train['work_begins_hour_mean_year']
train['3M/Y_work_begins_hour_std'] = train['work_begins_hour_std_3m']/train['work_begins_hour_std_year']
train['3M/Y_work_begins_hour_max'] = train['work_begins_hour_max_3m']/train['work_begins_hour_max_year']
train['3M/Y_work_begins_hour_min'] = train['work_begins_hour_min_3m']/train['work_begins_hour_min_year']

train['3M/Y_work_ends_hour_mean'] = train['work_ends_hour_mean_3m']/train['work_ends_hour_mean_year']
train['3M/Y_work_ends_hour_std'] = train['work_ends_hour_std_3m']/train['work_ends_hour_std_year']
train['3M/Y_work_ends_hour_max'] = train['work_ends_hour_max_3m']/train['work_ends_hour_max_year']
train['3M/Y_work_ends_hour_min'] = train['work_ends_hour_min_3m']/train['work_ends_hour_min_year']

train['3M/Y_work_duration_mean'] = train['work_duration_mean_3m']/train['work_duration_mean_year']
train['3M/Y_work_duration_std'] = train['work_duration_std_3m']/train['work_duration_std_year']
train['3M/Y_work_duration_max'] = train['work_duration_max_3m']/train['work_duration_max_year']
train['3M/Y_work_duration_min'] = train['work_duration_min_3m']/train['work_duration_min_year']

In [14]:
train['wtd_mean/wd_mean_3m'] = train['work_time_day_mean_3m']*train['work_time_day_mean_3m']
train['wtd_mean/wd_mean_year'] = train['work_time_day_mean_year']*train['work_time_day_mean_year']

train['wtd_std/wd_std_3m'] = train['work_time_day_std_3m']*train['work_time_day_std_3m']
train['wtd_std/wd_std_year'] = train['work_time_day_std_year']*train['work_time_day_std_year']

train['wtd_max/wd_max_1m'] = train['work_time_day_max_1m']*train['work_time_day_max_1m']
train['wtd_max/wd_max_3m'] = train['work_time_day_max_3m']*train['work_time_day_max_3m']
train['wtd_max/wd_max_year'] = train['work_time_day_max_year']*train['work_time_day_max_year']

train['wtd_min/wd_min_1m'] = train['work_time_day_min_1m']*train['work_time_day_min_1m']
train['wtd_min/wd_min_3m'] = train['work_time_day_min_3m']*train['work_time_day_min_3m']
train['wtd_min/wd_min_year'] = train['work_time_day_min_year']*train['work_time_day_min_year']

In [15]:
train['3M/Y_w/w_mean'] = train['wtd_mean/wd_mean_3m']/train['wtd_mean/wd_mean_year']
train['3M/Y_w/w_std'] = train['wtd_std/wd_std_3m']/train['wtd_std/wd_std_year']
train['3M/Y_w/w_max'] = train['wtd_max/wd_max_3m']/train['wtd_max/wd_max_year']
train['3M/Y_w/w_min'] = train['wtd_min/wd_min_3m']/train['wtd_min/wd_min_year']

In [16]:
len(valid_logins['tabnum'].unique())

33299

In [17]:
len(train['tabnum'].unique())

37902

In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 449576 entries, 0 to 449575
Columns: 476 entries, C_Year to 3M/Y_w/w_min
dtypes: datetime64[ns](1), float64(406), int64(62), object(7)
memory usage: 1.6+ GB


In [19]:
train.describe().T

,count,mean,std,min,25%,50%,75%,max
EtalonPosName,444778.0,3.349986e-04,2.566408e-02,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,3.000000e+00
age_qty,449537.0,2.933861e+01,6.587690e+00,1.800000e+01,2.500000e+01,2.800000e+01,3.300000e+01,6.200000e+01
boss_pos_name,443904.0,5.774041e-01,2.584809e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,7.400000e+01
boss_subord_1y_max_qty,443001.0,2.610123e+01,3.032693e+01,0.000000e+00,1.500000e+01,2.100000e+01,2.900000e+01,7.680000e+02
boss_subord_1y_min_qty,443001.0,1.800040e+01,1.663535e+01,0.000000e+00,1.000000e+01,1.600000e+01,2.300000e+01,5.170000e+02
boss_subord_in_1m_qty,443001.0,2.029451e+00,1.040976e+01,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,6.980000e+02
boss_subord_in_1y_qty,443001.0,1.368384e+01,3.958870e+01,0.000000e+00,3.000000e+00,7.000000e+00,1.400000e+01,2.021000e+03
boss_subord_out_1m_qty,443001.0,1.244683e+00,5.063841e+00,0.000000e+00,0.000000e+00,1.000000e+00,2.000000e+00,5.830000e+02
boss_subord_out_1y_qty,443001.0,1.145398e+01,3.154703e+01,0.000000e+00,2.000000e+00,6.000000e+00,1.300000e+01,1.970000e+03
boss_subord_qty,443001.0,2.364727e+01,2.518425e+01,0.000000e+00,1.400000e+01,1.900000e+01,2.700000e+01,7.680000e+02


In [20]:
train.describe(include=['object','bool']).T

,count,unique,top,freq
C_Year,327732,5,C,236023
R_Year,327587,5,C,199119
gender,449567,2,F,396672
lbt_payroll_min_dt,447701,1508,05.07.2012,58527
srv_sbol_atm_1st_login_ever_dt,316377,1277,21.01.2015,2563
srv_sbol_mob_1st_login_ever_dt,294304,1347,01.01.2015,24041
srv_sbol_web_1st_login_ever_dt,316021,1245,01.01.2015,11873


In [21]:
train['MY_year']= train['C_Year']+train['R_Year']

# preprocessing

In [22]:
import datetime as dt

In [23]:
def preprocess_dataset(df):
    print('Memory optimization...')
    for dtype in ['float','int','object']:
        selected_dtype = df.select_dtypes(include=[dtype])
        mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
        mean_usage_mb = mean_usage_b / 1024 ** 2
        print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb)) 
    
    print('Dates processing...')
    for column in [col for col in df if col.endswith('dt')]:
    
        df[column] = pd.to_datetime(df[column])
        
        NOW = dt.datetime(2019,6,24)
        df[column+"_DAY"]=df[column].apply(lambda x: x.day)
        df[column+"_MONTH"]=df[column].apply(lambda x: x.month)
        df[column+"_YEAR"]=df[column].apply(lambda x: x.year)
        df[column]=df[column].apply(lambda x: pd.to_timedelta(NOW-x).days) 
    
    print('Floats processing...')
    for every in df.select_dtypes(include=['float']).columns:
        df[every] = df[every].apply(pd.to_numeric,downcast='float')
    
    print('Cats processing...')   
    for every  in list(df.select_dtypes(include=['object']).columns):
        df = pd.concat([df,pd.get_dummies(df[every],prefix=every)],axis=1)
        df.drop(every,1,inplace=True)
        
    for dtype in ['float','int','object']:
        selected_dtype = df.select_dtypes(include=[dtype])
        mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
        mean_usage_mb = mean_usage_b / 1024 ** 2
        print("Average memory usage for {} columns: {:03.2f} MB".format(dtype,mean_usage_mb)) 

In [24]:
preprocess_dataset(train)   

Memory optimization...
Average memory usage for float columns: 3.42 MB
Average memory usage for int columns: 0.00 MB
Average memory usage for object columns: 21.74 MB
Dates processing...
Floats processing...
Cats processing...
Average memory usage for float columns: 3.42 MB
Average memory usage for int columns: 0.00 MB
Average memory usage for object columns: 0.00 MB


In [26]:
for every  in list(train.select_dtypes(include=['object']).columns):
        train = pd.concat([train,pd.get_dummies(train[every],prefix=every)],axis=1)
        train.drop(every,1,inplace=True)

MemoryError: 

In [ ]:
train=train.dropna(axis=1, how='all')

In [ ]:
train.drop(train.columns[train.apply(lambda col:col.value_counts().iloc[0]/len(col)>0.999)],axis=1,inplace=True)
train.shape

# tuning

# prediction

In [ ]:
import lightgbm as lgb

In [ ]:
d_train = lgb.Dataset(train.drop('target_churn_lag1m',1), label=train['target_churn_lag1m'])

params = {'class_weight':'balanced',
          'boosting_type': 'gbdt',
          'learning_rate': 0.01,
          'objective':'binary',
          'metric':'auc',
          'n_estimators':100,
          'num_leaves': 100,
          'max_depth':24,
          'min_data_in_leaf': 20,
          'max_bin': 510,
          'colsample_bytree' : 1,
            'subsample' : 0.75,
            'reg_alpha' : 1.2,
            'reg_lambda' : 1.2
         }

clf = lgb.train(params, d_train, 100)

In [ ]:
feats = pd.DataFrame(sorted(zip(clf.feature_importance(),clf.feature_name())), columns=['Value','Feature'])

In [ ]:
feats.iloc[:-10]

In [ ]:
test['report_dt'] = pd.to_datetime(test['report_dt'])

In [ ]:
logins4test =test.apply(aggregate_logins,1)
test = test.join(logins4test)

In [ ]:
test['3M/Y_work_time_day_mean'] = test['work_time_day_mean_3m']/test['work_time_day_mean_year']
test['3M/Y_work_time_day_std'] = test['work_time_day_std_3m']/test['work_time_day_std_year']
test['3M/Y_work_time_day_max'] = test['work_time_day_max_3m']/test['work_time_day_max_year']
test['3M/Y_work_time_day_min'] = test['work_time_day_min_3m']/test['work_time_day_min_year']

test['3M/Y_work_begins_hour_mean'] = test['work_begins_hour_mean_3m']/test['work_begins_hour_mean_year']
test['3M/Y_work_begins_hour_std'] = test['work_begins_hour_std_3m']/test['work_begins_hour_std_year']
test['3M/Y_work_begins_hour_max'] = test['work_begins_hour_max_3m']/test['work_begins_hour_max_year']
test['3M/Y_work_begins_hour_min'] = test['work_begins_hour_min_3m']/test['work_begins_hour_min_year']

test['3M/Y_work_ends_hour_mean'] = test['work_ends_hour_mean_3m']/test['work_ends_hour_mean_year']
test['3M/Y_work_ends_hour_std'] = test['work_ends_hour_std_3m']/test['work_ends_hour_std_year']
test['3M/Y_work_ends_hour_max'] = test['work_ends_hour_max_3m']/test['work_ends_hour_max_year']
test['3M/Y_work_ends_hour_min'] = test['work_ends_hour_min_3m']/test['work_ends_hour_min_year']

test['3M/Y_work_duration_mean'] = test['work_duration_mean_3m']/test['work_duration_mean_year']
test['3M/Y_work_duration_std'] = test['work_duration_std_3m']/test['work_duration_std_year']
test['3M/Y_work_duration_max'] = test['work_duration_max_3m']/test['work_duration_max_year']
test['3M/Y_work_duration_min'] = test['work_duration_min_3m']/test['work_duration_min_year']

In [ ]:
test['wtd_mean/wd_mean_3m'] = test['work_time_day_mean_3m']*test['work_time_day_mean_3m']
test['wtd_mean/wd_mean_year'] = test['work_time_day_mean_year']*test['work_time_day_mean_year']

test['wtd_std/wd_std_3m'] = test['work_time_day_std_3m']*test['work_time_day_std_3m']
test['wtd_std/wd_std_year'] = test['work_time_day_std_year']*test['work_time_day_std_year']

test['wtd_max/wd_max_3m'] = test['work_time_day_max_3m']*test['work_time_day_max_3m']
test['wtd_max/wd_max_year'] = test['work_time_day_max_year']*test['work_time_day_max_year']

test['wtd_min/wd_min_3m'] =test['work_time_day_min_3m']*test['work_time_day_min_3m']
test['wtd_min/wd_min_year'] = test['work_time_day_min_year']*test['work_time_day_min_year']

In [ ]:
test['3M/Y_w/w_mean'] = test['wtd_mean/wd_mean_3m']/test['wtd_mean/wd_mean_year']
test['3M/Y_w/w_std'] = test['wtd_std/wd_std_3m']/test['wtd_std/wd_std_year']
test['3M/Y_w/w_max'] = test['wtd_max/wd_max_3m']/test['wtd_max/wd_max_year']
test['3M/Y_w/w_min'] = test['wtd_min/wd_min_3m']/test['wtd_min/wd_min_year']

In [ ]:
test['MY_year']= test['C_Year']+test['R_Year']

In [ ]:
preprocess_dataset(test)
for every  in list(test.select_dtypes(include=['object']).columns):
        test = pd.concat([test,pd.get_dummies(test[every],prefix=every)],axis=1)
        test.drop(every,1,inplace=True)

In [ ]:
cols = list(train.columns)
cols.remove('target_churn_lag1m')

test = test[cols]

In [ ]:
test.shape

In [ ]:
y_test = clf.predict(test)

In [ ]:
pd.concat([test['tabnum'], pd.Series(y_test,name='prob')], axis=1, sort=False).to_csv('gaev_submission.csv')